In [1]:
import pandas as pd 
import statsmodels.api as sm
import joblib
predict_data = pd.read_csv("../../data/raw/predict_data.csv", index_col=None, header=0)
test_data = predict_data.copy()
test_data.head()

,market_id,created_at,store_id,subtotal,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_store_to_consumer_driving_duration,delivery_id
0,3.0,2015-02-25 02:22:30,5477,7500,4.0,4.0,4.0,670.0,194096
1,3.0,2015-02-25 01:14:19,5477,7100,4.0,1.0,1.0,446.0,236895
2,4.0,2015-02-22 02:27:44,5477,4500,9.0,7.0,6.0,504.0,190868
3,3.0,2015-02-25 01:32:39,5477,1700,3.0,3.0,3.0,687.0,183076
4,1.0,2015-02-24 05:20:45,2841,3150,4.0,4.0,4.0,528.0,186200


In [2]:
# test set preparation 
test_data['time_of_day_order_created_at'] = pd.to_datetime(test_data["created_at"]).dt.hour + pd.to_datetime(test_data["created_at"]).dt.minute/60
test_data['total_avaialable_dashers'] = test_data['total_onshift_dashers'] - test_data['total_busy_dashers']

In [3]:
test_data_columns = test_data[['time_of_day_order_created_at', \
                         'total_avaialable_dashers', 'total_outstanding_orders',\
                        'estimated_store_to_consumer_driving_duration']]
imputer = joblib.load("../tmp_models/{}_model.pkl".format('imputer'))
test_features = pd.DataFrame(imputer.fit_transform(test_data_columns),columns = test_data_columns.columns)

#test_data_columns = sm.add_constant(test_data_columns) #Removing the intercept term 

In [4]:
model = joblib.load("../tmp_models/{}_model.pkl".format('xgbr'))
predictions = model.predict(test_features)
predictions

array([2693.9429, 3057.6462, 2590.251 , ..., 2220.5942, 2209.4685,
       1830.4609], dtype=float32)

In [5]:
predict_data['predicted_duration'] = predictions
predict_data.head()

,market_id,created_at,store_id,subtotal,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_store_to_consumer_driving_duration,delivery_id,predicted_duration
0,3.0,2015-02-25 02:22:30,5477,7500,4.0,4.0,4.0,670.0,194096,2693.942871
1,3.0,2015-02-25 01:14:19,5477,7100,4.0,1.0,1.0,446.0,236895,3057.646240
2,4.0,2015-02-22 02:27:44,5477,4500,9.0,7.0,6.0,504.0,190868,2590.250977
3,3.0,2015-02-25 01:32:39,5477,1700,3.0,3.0,3.0,687.0,183076,3113.440430
4,1.0,2015-02-24 05:20:45,2841,3150,4.0,4.0,4.0,528.0,186200,3406.943115


In [6]:
predict_data.to_csv("../../data/predicted/data_to_predict.csv")